In [4]:
%pip install --upgrade -q aiobotocore
%pip install -q  xgboost==1.3.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.26.111 requires botocore<1.30.0,>=1.29.111, but you have botocore 1.29.76 which is incompatible.
awscli 1.27.111 requires botocore==1.29.111, but you have botocore 1.29.76 which is incompatible.
awscli 1.27.111 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.111 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install --upgrade boto3 botocore

  Using cached boto3-1.26.154-py3-none-any.whl (135 kB)
  Using cached botocore-1.29.154-py3-none-any.whl (10.9 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.111
    Uninstalling boto3-1.26.111:
      Successfully uninstalled boto3-1.26.111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.154 which is incompatible.
awscli 1.27.111 requires botocore==1.29.111, but you have botocore 1.29.154 which is incompatible.
awscli 1.27.111 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
sagemaker 2.145.0 requires PyYAML==5.4.1, but you have pyyaml 5.3 which is incompatible.
Not

In [5]:
%pip install --upgrade pip


  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# Set SageMaker and S3 client variables
sess = sagemaker.Session()

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)

sagemaker_role = sagemaker.get_execution_role()

# Set read and write S3 buckets and locations
write_bucket = sess.default_bucket()

read_bucket = "sagemaker-sample-filesyash"

train_data_key = f"train.csv"
test_data_key = f"test.csv"
model_key = f"model"
output_key = f"output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"

In [8]:
hyperparams = {
                "max_depth": 3,
                "eta": 0.2,
                "objective": "binary:logistic",
                "subsample" : 0.8,
                "colsample_bytree" : 0.8,
                "min_child_weight" : 3
              }

num_boost_round = 100
nfold = 3
early_stopping_rounds = 10


In [ ]:

# Set up data input
label_col = "LABEL_ENCODED"
data = pd.read_csv(train_data_uri)
del data['Unnamed: 0']

In [10]:
data.columns

Index(['RATING', 'VERIFIED_PURCHASE', 'REVIEW_LENGTH', 'TITLE_LENGTH',
       'NUM_NOUNS', 'NUM_ADJECTIVES', 'NUM_ADVERBS', 'NUM_VERBS',
       'AVG_WORD_LENGTH', 'COHERENT_ENCODED', 'READABILITY_FRE',
       'RATING_DEVIATION', 'LABEL_ENCODED'],
      dtype='object')

In [11]:

# Recolumnstraining data and target
train_features = data.drop(label_col, axis=1)
train_label = pd.DataFrame(data[label_col])
dtrain = xgb.DMatrix(train_features, label=train_label)

# Cross-validate on training data
cv_results = xgb.cv(
    params=hyperparams,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    nfold=nfold,
    early_stopping_rounds=early_stopping_rounds,
    metrics=["auc"],
    seed=10,
)


metrics_data = {
    "binary_classification_metrics": {
        "validation:auc": {
            "value": cv_results.iloc[-1]["test-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["test-auc-std"]
        },
        "train:auc": {
            "value": cv_results.iloc[-1]["train-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["train-auc-std"]
        },
    }
}


print(f"Cross-validated train-auc:{cv_results.iloc[-1]['train-auc-mean']:.2f}")
print(f"Cross-validated validation-auc:{cv_results.iloc[-1]['test-auc-mean']:.2f}")

Cross-validated train-auc:0.87
Cross-validated validation-auc:0.86


In [12]:
data = pd.read_csv(test_data_uri)
del data['Unnamed: 0']

In [13]:
test_features = data.drop(label_col, axis=1)
test_label = pd.DataFrame(data[label_col])
dtest = xgb.DMatrix(test_features, label=test_label)

model = (xgb.train(params=hyperparams, dtrain=dtrain, evals = [(dtrain,'train'), (dtest,'eval')], num_boost_round=num_boost_round, 
                  early_stopping_rounds=early_stopping_rounds, verbose_eval = 0)
        )

# Test model performance on train and test sets
test_pred = model.predict(dtest)
train_pred = model.predict(dtrain)

test_auc = roc_auc_score(test_label, test_pred)
train_auc = roc_auc_score(train_label, train_pred)

print(f"Train-auc:{train_auc:.2f}, Test-auc:{test_auc:.2f}")

[06:39:16] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train-auc:0.88, Test-auc:0.86


In [14]:
#PRINTING THE ACCURACY AND CLASSIFICATION REPORT
from sklearn.metrics import accuracy_score, classification_report

In [20]:
predictions = test_pred > 0.5

In [21]:
print(classification_report(test_label, predictions))

              precision    recall  f1-score   support

           0       0.86      0.71      0.77      2115
           1       0.75      0.88      0.81      2085

    accuracy                           0.79      4200
   macro avg       0.80      0.79      0.79      4200
weighted avg       0.80      0.79      0.79      4200



In [22]:
test_accuracy = 100*accuracy_score(test_label, predictions)
print(f"Accuracy: {test_accuracy}%")

Accuracy: 79.21428571428571%


In [23]:
predictions

array([False,  True, False, ...,  True,  True,  True])

In [24]:
test_label

,LABEL_ENCODED
0,0
1,1
2,0
3,1
4,1
...,...
4195,1
4196,1
4197,1
4198,0
